[컨볼루션 신경망 레이어 이야기](https://tykimos.github.io/2017/01/27/CNN_Layer_Talk/)  
하기 내용은 상기 링크를 정리한것.

# 컨볼루션 신경망 레이어 이야기

## 필터로 특징을 뽑아주는 컨볼루션(Convolution) 레이어
케라스에서 제공되는 컨볼루션 레이어 종류에도 여러가지가 있으나 영상 처리에 주로 사용되는 Conv2D 레이어를 살펴본다.  
레이어는 영상인식에 주로 사용되며, 필터가 탑재된다.  
아래는 그에 따른 Conv2D클래스 사용예제 이다.

```
Conv2D(32, (5, 5), padding='valid', input_shape=(28, 28, 1), activation='relu')
```

### 주요 인자
* 첫번째 인자 : '32' 컨볼루션 필터의 수이다.
* 두번째 인자 : '(5,5)' 컨볼루션 커널의 (행, 열)이다.
* padding : 경계 처리 방법을 정의한다.
    - 'valid' : 유효한 영역만 출력이 된다. 따라서 출력 이미지 사이즈는 입력 사이즈 보다 작다.
    - 'same'  : 출력 이미지 사이즈가 입력 이미지 사이즈와 동일하다.
* input_shape : 샘플 수를 제외한 입력 형태를 정의한다.  
모델에서 첫 레이어 일때만 정의한다.  
    - (행, 열, 채널 수)로 정의한다.  
    흑백영상인 경우 채널이 **1**이고, 컬러(RGB)영상인 경우 채널을 **3**으로 설정한다.
* activation : 활성화 함수 설정을 한다.
    - 'linear' : 디폴트 값, 입력뉴런과 가중치로 계산된 결과값이 그대로 출력된다.
    - 'relu' : rectifier함수, 은닉층에 주로 사용된다. 
    - 'sigmoid' : 시그모이드 함수, 이진 분류 문제에서 출력층에 주로 쓰인다.  
    - 'softmax' : 소프트맥스 함수, 다중 클래스 분류 문제에서 출력층에 주로 쓰인다.  
    

### 입력 형태
* image_data_format이 'channels_first'인 경우 (샘플 수, **채널 수**, 행, 열)로 이루어진 4D 텐서이다.
* image_data_format이 'channels_last'인 경우 (샘플 수, 행, 열, **채널 수**)로 이루어진 4D 텐서이다.  
image_data_format은 "keras.json" 파일 안에 있는 설정이다.  
콘솔에서 "vim ~/.keras/kreas.json"으로 keras.json파일 내용을 변경할 수 있다.  
<br>

### 출력 형태
* image_data_format이 'channels_first'인 경우 (샘플 수, **필터 수**, 행, 열)로 이루어진 4D 텐서를 말한다. 
* image_data_format이 'channels_last'인 경우(샘플 수, 행, 열 **필터 수**)로 이루어진 4D 텐서를 말한다.
* 행과 열의 크기가 padding가 'same'인 경우에는 입력 형태의 행과 열의 크기와 동일하다.

간단한 예제로 컨볼루션 레이어와 필터에 대해 알아본다.  
* 입력 이미지는 채널 수가 1, 너비가 3 픽셀, 높이가 3픽셀인 데이터이고 크기가 $2 \times 2$ 인 필터가 하나인 경우 레이어로 표시하면 다음과 같다.  
단, 아래 예제는 image_data_format이 'channels_last'인 경우이다.
  

```
Conv2D(1, (2, 2), padding= 'valid', input_shape=(3, 3, 1))
```

이를 도식화 하면 아래와 같다.<br>
<br>
![](http://tykimos.github.io/warehouse/2017-1-27_CNN_Layer_Talk_lego_1.png)

<br>
필터는 가중치를 의미한다. 하나의 필터(w)가 입력 이미지(x)를 순회하면서 적용된 결과값을 모으면 출력이미지가 생성된다.(y)  
여기에는 두 가지 특성이 있다.
* 하나의 필터로 입력 이미지를 순회할 때 적용되는 가중치는 모두 동일하다.  
이를 파라미터 공유라고 부르며, 이 것으로 인하여 학습해야 하는 가중치의 수를 현저하게 줄여준다.
  <br><br>
* 출력에 영향을 미치는 영역이 지역적으로 제한된다.  
즉, 그림 $y_0$에 영향을 미치는 입력은 $x_0, x_1, x_3, x_4$ 데이터들로 한정된다.  
다시말해 코를 볼때는 코 주변만 보고, 눈을 볼 때는 눈 주변만 보면서 학습 및 인식하는 것을 말한다. 

### 가중치의 수
이를 Dense레이어와 컨볼루션 레이어와 비교를 해보면서 차이점을 알아본다.   
영상도 결국에는 픽셀의 집합이므로 입력 뉴런이 9개 ($3 \times3$)이고,  
출력 뉴런이 4개 ($2 \times 2$)인 Dense레이어로 표현될수 있다.
```
Dense(4, input_dim=9)
```
이를 도식화 하면 아래 그림과 같다.<br>
<br>
![](http://tykimos.github.io/warehouse/2017-1-27_CNN_Layer_Talk_lego_2.png)

가중치 (w) 즉, 시냅스 강도는 녹색 블럭으로 표시되어 있다.  
컨볼루션 레이어에서 가중치 4개로 구성된 크기가 $2 \times 2$인 필터를 적용하였을 때의 뉴런 상세 구조는 다음과 같다.  

![](http://tykimos.github.io/warehouse/2017-1-27_CNN_Layer_Talk_lego_3.png)

필터가 지역적으로만 적용되어 출력 뉴런에 영향을 미치는 입력 뉴런이 제한적이므로 Dense레이어와 비교했을 때,  
가중치가 많이 줄어든 모습을 볼 수 있다.  
게다가 녹색 블럭 상단에 표시된 빨간색, 파란색, 분홍색, 노란색끼리는 모두 동일한 가중치(파라미터를 공유)이므로 결국 가용되는 가중치는 4개 이다.  
즉, Dense 레이어에서는 36개의 가중치가 사용되었지만, 컨볼루션 레이어에서는 필터의 크기인 4개의 가중치만을 사용한다.  

## 경계 처리 방법
이번에는 경계 처리 방법에 대해서 알아본다.  
컨볼루션 레이어 설정 옵션에는 border_mode가 있는데,  
'valid'와 'same'으로 설정이 가능하다.  
이 둘의 차이는 아래의 그림을 보면 더 확실하게 알 수 있다.
![](http://tykimos.github.io/warehouse/2017-1-27_CNN_Layer_Talk_lego_4.png)

* 'valid' : 상단 이미지를 보면 입력 이미지 영역에 맞게 필터를 적용하기 때문에 출력 이미지 크기가 입력 이미지 크기보다 작아진다.  <br>
<br>
* 'same'  : 하단 이미지를 보면 반면에 출력 이미지와 입력 이미지 사이즈가 동일하도록 입력 이미지 경계에 빈 영역을 추가하여 필터를 적용한다.   
'same'으로 설정 시, 입력 이미지에 경계를 학습시키는 효과가 있다.

## 필터 수
다음은 필터의 개수에 대해서 알아본다.  
입력 이미지가 단채널의 $3 \times 3$이고, $2 \times 2$인 필터가 하나 있다면 다음과 같이 컨볼루션 레이어를 정의할 수 있다.

```
Conv2D(1, (2, 2), padding = 'same', input_shape=(3, 3, 1)
```

이를 도식화 하면 다음과 같다. 
![](http://tykimos.github.io/warehouse/2017-1-27_CNN_Layer_Talk_lego_5.png)

만약 여기서 $2 \times 2$ 필터를 3개 사용한다면 다음과 같이 정의할 수 있다.

```
Conv2D(3, (2, 2), padding='same', input_shape=(3, 3, 1))
```
이를 도식화하면 다음과 같다.
![](http://tykimos.github.io/warehouse/2017-1-27_CNN_Layer_Talk_lego_6.png)

여기서 살펴봐야할 것은 필터가 3개라서 출력 이미지도 필터 수에 따라 3개로 늘어났다.  
총 가중치의 수는 $3 \times 2 \times 2$으로 12개이다.  
필터마다 고유한 특징을 뽑아 고유한 출력 이미지로 만들기 때문에 필터의 출력값을 더해서 하나의 이미지로 만들거나 그렇게 하지 않는다.  
만약 필터에 대해 생소하다면 아래 스마트폰 카메라로 사진을 찍었을 때 필터를 적용해볼 수 있다.  
이때 적용되는 필터에 따라 다른 사진이 나오는 것을 볼 수 있다.  
![](http://tykimos.github.io/warehouse/2017-1-27_CNN_Layer_Talk_filter.png)

뒤에서 각 레이어를 레고처럼 쌓아 올리기 위해 약식으로 표현하면 다음과 같다.  
![](http://tykimos.github.io/warehouse/2017-1-27_CNN_Layer_Talk_lego_7.png)

이 표현은 다음과 같다. 
* 입력 이미지 사이즈가 $3 \times 3$ 이다.
* $2 \times 2$ 커널을 가진 필터가 3개이다. 가중치는 총 12개 이다.  
* 출력 이미지 사이즈가 $3 \times3$이고 총 3개이다. 이는 채널이 3개 다라고도 표현한다.